<br>
<br>
<br>
<br>

# Do the same project(pipeline- (01 & 02))

<br>
<br>
<br>
<br>


In [21]:

import pandas as pd 
import numpy as np 
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier



In [11]:
df = pd.read_csv("dataset/train.csv")
df.sample(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
869,870,1,3,"Johnson, Master. Harold Theodor",male,4.0,1,1,347742,11.1333,NaN,S
372,373,0,3,"Beavan, Mr. William Thomas",male,19.0,0,0,323951,8.0500,NaN,S


In [12]:
df.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)
df.sample(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
436,0,3,female,21.0,2,2,34.375,S
272,1,2,female,41.0,0,1,19.500,S


In [17]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

<br>

## 01. train_test_split()

In [18]:

X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],df["Survived"],test_size=0.2,random_state=42)

X_train.sample(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
87,3,male,NaN,0,0,8.05,S
53,2,female,29.0,1,0,26.00,S


In [19]:

# imputation transformer:

trnf1 = ColumnTransformer(
    transformers=[
        ('impute_age',SimpleImputer(strategy="mean"),[2]), # instead of giving column name give number index:
        ('impute_embarked',SimpleImputer(strategy="most_frequent"),[6])
    ],
    remainder='passthrough'
)

In [20]:

# One-Hot_Encoding Transformer:

trnf2 = ColumnTransformer(
    transformers=[
        ("ohe_sex_embarked",OneHotEncoder(sparse_output=False,handle_unknown="ignore"),[1,6])
    ],
    remainder="passthrough"
)


In [22]:
#(in this we use feature selection that why we need to use minmaxscaler)
# scaling:
trnf3 = ColumnTransformer(
    transformers=[
        ('scale',MinMaxScaler(),slice(0,10)) # after applying trn1,trns1 we have total 9 columns:
    ],
    remainder="passthrough"
)

<br>

# feature selection:

<br>

In [23]:
from sklearn.feature_selection import SelectKBest,chi2

In [27]:

trnf4 = SelectKBest(score_func=chi2,k=5)  #  (k = 5 top 5 important columns:)


In [31]:
# train the model: 

trnf5 = DecisionTreeClassifier()

<br>
<br>
<br>

# Create Pipelines:

<br>
<br>
<br>

In [29]:
from sklearn.pipeline import Pipeline,make_pipeline

In [32]:
pipe = Pipeline(
    # transformer name, and hers object name
    [
        ('trnf1',trnf1),
        ('trnf2',trnf2),
        ('trnf3',trnf3),
        ('trnf4',trnf4),
        ('trnf5',trnf5),
     ]
)

In [34]:
# we can also do the same thing with make_pipeline:

pipe  = Pipeline((trnf1,trnf2,trnf3,trnf4,trnf5))

In [35]:
# train the pipe line:

pipe.fit(X_train,y_train)


TypeError: ColumnTransformer is subscriptable after it is fitted